# Projet 3 (LU3IN005) : Chaines de Markov et épidémiologie :
# Propagation d’une épidémie dans une population

### <u>Groupe 6 :</u>
<pr>Samy (Adem) YAZID<br>Numéro étudiant : $3811684$<br><br>Margaux MARSELOO<br>Numéro étudiant : $28630611$</pr> 

### <u>**Consignes :**</u>
<pr>

L’objectif de ce projet est de manipuler des chaînes de Markov pour étudier la propagation d’une épidémie dans une population. Votre rendu sera un notebook, il est attendu que les codes soient commentés
et les résultats interprétés. Les packages <em>numpy</em>, <em>random</em> et <em>matplotlib</em> sont conseillés.
Nous allons étudier des populations constituées de $3$ types d’individus. Chaque individu est dans un
des $3$ états suivant : sain $S$, infecté $I$ ou guéri $R$.
</pr>

## I) <u>Apprentissage des paramètres d’un modèle à partir de données</u>

<pr>
    
Dans notre modèle, nous allons considérer qu’à chaque temps :
- Chaque individu sain peut rester sain ou devenir infecté
- Chaque individu infecté peut rester infecté ou devenir guéri
- Chaque individu guéri reste guéri
et que la probabilité de passer d’un état à l’autre ne dépend que de l’état précédent.

Nous disposons d’une séquence d’observations et nous souhaitons apprendre les paramètres de la
chaîne de Markov permettant de modéliser le processus sous-jacent qui a généré la séquence.
Nous avons suivi un individu pendant $10$ jours, afin de déterminer a chaque temps dans quel état se trouvait l’individu.
Nous avons obtenu la séquence d’observation suivante : $S, S, S, I, I ,I ,I , I, I, R$.
</pr>


<pr><u>**Question 1:**</u> À partir de cette séquence d’observation, estimez de les probabilités de transition entre les états et dresser la matrice de probabilité de transitions.</pr>


<pr><u>**Question 2:**</u>
1) Lire des données
2) Estimez les probabilités de transition entre les états et dressez la matrice de probabilité de transitions
(indice pour vérifier vos résultats : la première ligne de la matrice est $[0.9308, 0.0691, 0.]$).
</pr>

## II) <u>Description du premier modèle</u>
<pr>La probabilité pour un individu d’être dans un de ces $3$ états au temps $t$, ne dépend que l’état dans
lequel il est au temps $t − 1$.
Un individu dans l’état sain a une probabilité de $0.92$ de rester sain et une probabilité de $0.08$ de
devenir infecté. Si l’individu est infecté, il peut le rester avec une probabilité de $0.93$ et être guéri avec
une probabilité de $0.07$. S’il est dans l’état guéri, il reste dans cet état avec une probabilité de $1$.

<u>**Question 1:**</u> À partir du graphe de transition, créez la matrice de transition $A$, la matrice contenant les probabilités de transition entre les différents états. Créez une fonction permettant de vérifier qu’une matrice est stochastique.
</pr>

<pr>
Au temps $t = 0$, un individu a une probabilité de $0.9$ d’être sain et $0.1$ d’être infecté.

<u>**Question 2:**</u> Créez $π_0$ la distribution de probabilité initiale.
</pr>

<pr>
Nous allons étudier l’évolution du nombre d’individu sains, infectés et guéris au cours du temps.
Dans un premier temps nous étudierons la distribution théorique, puis la distribution observée sur des simulations.
</pr>

### <u>Distribution $π_t$<u>

<pr><u>**Question 1:**</u> En utilisant $π_0$ et $A$, donnez la probabilité pour un individu d’être sain, infecté ou guéri au temps $t = 1$ (faire d’abord le calcul à la main).
</pr>


<pr><u>**Question 2:**</u> Donnez la probabilité pour un individu d’être sain, infecté ou guéri au temps $t = 2$ (faire d’abord le calcul à la main).
</pr>

<pr><u>**Question 3:**</u> De même pour chaque temps $t$ entre $1$ et $200$, calculez la distribution théorique des effectifs dans chaque état (<em>Rappel:</em> $π_t + 1 = π_t . A$ ).
</pr>

<pr><u>**Question 4:**</u> Représentez graphiquement la probabilité d’être dans chaque état en fonction du temps. (+ décrivez un peu ce que vous observez).
</pr>

### <u>Tirage aléatoire des états</u>
<pr>Vous allez générer une séquence de taille $T$ en utilisant cette chaîne de Markov. Pour générer une séquence aléatoire, choisissiez un état initial au hasard (en utilisant $π_0$) ; puis choisissez les états suivants en suivant les probabilités de transition <em>(= la matrice de transition $A$).</em>
Vous pouvez prendre $T=150$.
</pr>

### <u>Modélisation d’une population</u>
<pr>Vous avez généré une séquence d’état pour un individu. Maintenant vous allez générer un ensemble de séquences pour une population de 200 individus.

<u>**Question 1:**</u> À chaque temps $t$, comptez le nombre d’individus sains, infectés et guéris dans la population et affichez l’évolution du nombre d’individus dans les trois états en fonction du temps.
</pr>

<pr><u>**Question 2:**</u> Affichez le pourcentage d’individus sains infectés et guéris en fonction du temps.
</pr>

<pr><u>**Question 3:**</u> Quand $t$ est grand, quelle est la proportion d’individus sains, infectés et guéris ?
</pr>

<pr><u>**Question 4:**</u> Refaites les questions précédentes avec des populations de tailles différentes, $5$ individus et $5000$ individus par exemple.
</pr>

### <u>Longueur de l’infection</u>
<pr>Vous avez généré une séquence d’état pour un individu. Maintenant vous allez générer un ensemble de séquences pour une population de 200 individus.

<u>**Question 1:**</u> À partir des simulations, estimez la longueur moyenne d’une séquence de $I$.
</pr>

<u>**Question 2:**</u> Afficher la distribution observée de la longueur d’infection (peut-être que pour représenter une distribution, ça serait une bonne idée de faire un histogramme).
</pr>

<u>**Question 3:**</u> Calculez la longueur théorique d’une séquence de $I$ (vous pourrez utiliser l’espérance de la loi géométrique en justifiant).
</pr>

<u>**Question 4:**</u> Comparer la longueur estimée et la longueur théorique.
</pr>

<u>**Question 5:**</u> Affichez la distribution théorique de la longueur d’infection.
</pr>

### <u>Petites modifications autour de ce premier modèle</u>
<pr>Vous pouvez maintenant modifier le modèle pour étudier différents cas de figure, en
faisant varier certains paramètres :
</pr>

<u>**Question 1:**</u> Faire varier la taille de la population.
</pr>

<u>**Question 2:**</u> Faire varier la distribution de probabilité initiale.
</pr>

<u>**Question 3:**</u> Faire varier les probabilités de transition.
</pr>

## III) <u>Description du second modèle</u>
<pr>Nous allons maintenant considérer un second modèle, les individus guéris peuvent redevenir sains avec une probabilité de $0.02$. Ils-elles peuvent perdre leur immunité face à la maladie.
</pr>

### <u>Analyse du modèle</u>

<pr>

<u>**Question 1:**</u> Ce processus peut-il être modélisé par une chaîne de Markov ?
</pr>

<pr>

<u>**Question 2:**</u>
Donnez la nouvelle matrice de transition.
</pr>

<pr>
    
<u>**Question 3:**</u> Quels est la nature des états de cette chaîne de Markov ? Est-elle périodique ? Est-elle irréductible ?
</pr>

<pr>
    
<u>**Question 4:**</u> Calculez la matrice $A × A$. À quoi correspond-elle ? Est elle stochastique ? Même question pour $A^3$ et $A^4$.
</pr>

<pr>
    
<u>**Question 5:**</u> Réalisez les nouvelles simulations. Comment la population évolue-t-elle ?
</pr>

<pr>
    
<u>**Question 6:**</u> Refaites les simulations avec une autre distribution de probabilité initiale (par exemple si au temps $t = 0$, nous avons $90%$ d’infectés et $10%$ de sains). Explorez d’autres initialisations et comme pour chaque question commentez vos observations.
</pr>

<pr>
    
<u>**Question 7:**</u> Calculez la distribution de probabilité stationnaire et comparez ce résultat avec les simulations pour
$t$ assez grand.
</pr>

### <u>Longueur de l’immunité</u>

<pr>
    
On peut se demander combien de temps un individu qui a été malade, reste protégé contre la maladie.
    
<u>**Question 1:**</u> À partir des simulations, estimez la longueur moyenne d’une séquence de $R$.
</pr>

<pr>
    
<u>**Question 2:**</u> Calculez la longueur théorique d’une séquence de $R$.
</pr>

<pr>
    
<u>**Question 3:**</u> Affichez la distribution théorique et la distribution observée de la longueur de l’immunité.
</pr>

### <u>Modifier le modèle</u>

<pr>
   
<u>**Question 1:**</u> Comment l’épidémie évolue-t-elle si vous modifiez la probabilité pour un individu sain de devenir infecté ? Quelle est la nouvelle distribution a l’équilibre ?
</pr>

<pr>
    
<u>**Question 2:**</u> Calculez la longueur théorique d’une séquence de $R$.
</pr>

## IV) <u>Confinement</u>
<pr>On peut imaginer que si des mesures de distanciation sociale sont mises en place, la probabilité de devenir infecté devient nulle.

Nous allons alterner entre les périodes de non distanciation et de distanciation :
- En période de non-confinement, nous utilisons la matrice de transition de l’exercice précédent
- En période de confinement, la probabilité de devenir infecté pour un individu sain devient nulle

<u>**Question 1:**</u> Commencez les simulations avec la matrice de transition de l’exercice précédent.
- On peut considérer qu’au temps initial tous les individus sont sains.
- Quand il y a $25%$ d’individus infectés dans la population, nous passons en période de distanciation. Le nombre d’individus infectés va décroître.
- Quand il y a moins de $10%$ d’infectés, le confinement est levé.
</pr>

<pr>
    
<u>**Question 2:**</u> Faites les simulations pour une population assez grande, représentez l’évolution du nombre d’individus à chaque temps $t$ (vous devriez voir des “vagues”), et notez les temps de confinement et de deconfinement.
</pr>

<pr>
    
<u>**Question 3:**</u> Combien de confinements / déconfinements sont nécessaires ?
</pr>

## V) <u>Discussion</u>

<pr>

<u>**Question 1:**</u> Quelles remarques critiques pouvez faire sur les modèles utilisés ?
</pr>

<pr>
    
<u>**Question 2:**</u> <em>(Optionnelle)</em> Proposez des améliorations et implémentez-les si possible.
</pr>